In [ ]:
import pandas as pd
import numpy as np
import re
import os, sys
pd.set_option('display.max_rows', 800)

# 主文切割

In [ ]:
# 讀原檔在的資料夾
dirs = os.listdir('./totalText')

# 要存文檔的list
listData = []

# 主文的結構一定是"主文@#$%%^。"，所以擷取主文就是要把"主文"跟"。"之間的文字擷取出來
# 要用到三個正規化，第一個跟第二個是分割主文用，第三個是擷取csv檔名 
regex = r'主文'
regex2 = r'。'
regex3 = r'[0-9]+'

# 讀每個txt檔
for file in dirs:
    strText=""
    title = re.match(regex3,file)[0] # 擷取檔名
    path = f'./totalText/{title}.txt'
    f = open(path, 'r', encoding='UTF-8') # 讀檔
    text = f.read()
    f.close()
    jugText = re.split(regex, text) # 第一個正規化先把"主文"前後切開
    foreword = jugText[0] # 主文前的文字會是判決基礎資料，存在foreword
    del jugText[0] # 把主文前的文字刪除
    nextText = '主文'.join(jugText) # 因為有些文章在後文還會出現"主文"，所以這邊需要join起來維持後面文章的完整性
    jugText_2 = re.split(regex2, nextText) # 第二個正規化用"。"切開所有段落
    labelText = jugText_2[0] # 擷取後文的第一個句號前的文字
    del jugText_2[0] # 刪掉主文
    featureText = '。'.join(jugText_2) # 把後文join
    
    listData.append([title,foreword,labelText,featureText]) # 存到list



In [ ]:
print(len(listData))

In [ ]:
listData[7][2]

# 中文字轉數字

In [ ]:
transList = listData.copy()
# 輸入中數替換表
df = pd.read_csv('./chiToNum.csv',header=None,sep=',')
# 把DataFrame轉成Numpy形式再轉成List
tmpNu = np.array(df)
numList = tmpNu.tolist()

In [ ]:
jugDataFrame1[3][0]

In [ ]:
# before
# jugDataFrame1 = pd.DataFrame(listData) # 這一行做過一次就要註解起來，目的是要保留轉換前的文檔
jugDataFrame1[:]

In [ ]:
jugDataFrame1[27117:]

In [ ]:
# 判決list的內容逐筆做文字數字轉換
for i in range(len(transList)):
    for j in range(len(numList)):
        transList[i][1] = transList[i][1].replace(numList[j][0],str(numList[j][1]))
        transList[i][2] = transList[i][2].replace(numList[j][0],str(numList[j][1]))
        transList[i][3] = transList[i][3].replace(numList[j][0],str(numList[j][1]))

In [ ]:
# after
jugDataFrame2 = pd.DataFrame(transList)
jugDataFrame2[27117:]

In [ ]:
# 將第一次切割好的檔案輸出
jugDataFrame2.to_csv('./splitText.csv', index=False, header=False, encoding='utf_8_sig')

In [ ]:
# 把主文前一欄不必要的欄位刪掉，新增一個叫jpd的dataFrame
jpd = jugDataFrame2.drop([1], axis=1)
jpd.head()

In [ ]:
# 空值大部分是司法程序公告，因此補值
jpd[2]=jpd[2].replace(to_replace="", value="司法程序公告")
# check
# jpd[27117:]

In [ ]:
len(jpd)

In [ ]:
# 提取主文內容做成標籤
# regex提取主文內有XX天、XX元的元素；regex2是將沒有裁罰內容的判決盡量分類
regex = r'[\d]+日|[\d]+天|[\d]+元'
regex2 = r'無罪|撤銷|更正|不受理|駁回|管轄錯誤|停止審判|停止審理|移送|免刑|免訴|再開辯論|刪除|廢棄|停止訴訟|附表|簡易判決|簡易審判|補正|應增列|應補充|通常程序|簡式審判|應於本裁定送達|判決如主文'

labelList = []
for i in range(len(jpd)):
    # 不符regex、regex2的內容屬於無法歸類的文字直接貼上
    if (re.search(regex2,jpd[2][i]) or re.search(regex,jpd[2][i])) == None: 
        labelList.append(jpd[2][i])      
    # 不屬於regex2就表示符合regex，可以提取XX元、XX日、XX天的訊息，提取出的訊息的第一筆通常就是裁判結果
    elif re.search(regex2,jpd[2][i]) == None:
        labelList.append(re.search(regex,jpd[2][i])[0])
    # 剩下表示屬於regex，可以進行分類，提取出的訊息的第一筆通常就是分類結果
    else:
        labelList.append(re.search(regex2,jpd[2][i])[0])

print(len(labelList))
print(labelList)

In [ ]:
# 第一次做中數轉換表的時候會有很多沒轉成功的文字，沒轉成功的儲存格通常還是會有轉到1日、1天這種狀況
# 因此檢查轉出來的labList是否有提取到1日，可以check沒轉成功的中文數字是那些
for i in range(len(labelList)):
    if labelList[i] == "818日":
        print(i,jpd[2][i])


In [ ]:
# 少部分欄位無法用程式轉成功只能人工判讀
# labelList[20556] = "3000元"
# labelList[20867] = "3000元"
# labelList[22706] = "3日"
# labelList[23751] = "90日"
# labelList[24784] = "3000元"
# labelList[25020] = "司法程序公告"
# labelList[30033] = "司法程序公告"
# labelList[19283] = "100日"
labelList[29730] = "88日"

In [ ]:
# 把擷取出的判決結果添加上去
jpd[1] = labelList
jpd

In [ ]:
# 確認判決結果有幾種
len(jpd[1].value_counts())

In [ ]:
# 確認每個判決結果的筆數，小筆數如果可以分類就回到# 提取主文內容那一格的regex2中新增分類
jpd[1].value_counts()

In [ ]:
# 確認特定分類結果的判決內容是如何
print (jpd.loc[jpd[1] == "應於本裁定送達"])

In [ ]:
# 確認轉換前的原文
jugDataFrame1[2][26411]

In [ ]:
# 把初步提取出來的標籤做進一步的分類

# 這用於分類屬於司法程序公告的對象，不屬於這類別的都歸類成司法程序公告
regexTest = r'[\d]+日|[\d]+天|[\d]+元|無罪|撤銷|不受理|駁回|免刑|免訴|附表|判決如主文' 
# 做數字提取
regex0 = r'[\d]+'
# 分類屬於天數的對象
regex1 = r'[\d]+日|[\d]+天'
# 分類已有金額的對象
regex2 = r'[\d]+元'
# 可能可以作為後續分析的判決類別
regex3 = r'無罪|撤銷|不受理|駁回|免刑|免訴'
# 這類對象判決內容寫在後面，無法用目前方法提取出來
regex4 = r'附表|判決如主文'

cleanLabelList = []
for i in range(len(jpd)):
    # 不屬於regexTest的都歸類成司法程序公告
    if re.search(regexTest,jpd[1][i]) == None:
        cleanLabelList.append("司法程序公告")      
    # 不屬於regex1~3的屬於判決結果寫在後面，需要再去看的
    elif (re.search(regex1,jpd[1][i]) or re.search(regex2,jpd[1][i]) or re.search(regex3,jpd[1][i])) == None:
        cleanLabelList.append("判決待查")
    # 不屬於regex1~2的為文字判決類別，直接提取關鍵字
    elif (re.search(regex1,jpd[1][i]) or re.search(regex2,jpd[1][i])) == None:
        cleanLabelList.append(jpd[1][i])
    # 不屬於regex2的是判決天數，將數字提取出來後乘1000作為罰款金額
    elif re.search(regex2,jpd[1][i]) == None:
        days = re.search(regex0,jpd[1][i])[0]
        payMoney = int(days) * 1000
        cleanLabelList.append(payMoney)
    # 剩下判決為罰款的資料，直接擷取數字部分即可
    else:
        payMoney = int(re.search(regex0,jpd[1][i])[0])
        cleanLabelList.append(payMoney)

print(len(cleanLabelList))
print(cleanLabelList)

In [ ]:
# 新增進一步提取過的label到jpd上
jpd[4] = cleanLabelList
jpd

In [ ]:
# 計算剩幾種label
len(jpd[4].value_counts())

In [ ]:
# 看每個label的數量
jpd[4].value_counts()

In [ ]:
# 刪掉不必要的欄位，放到jpd2
jpd2 = jpd.drop([1,2], axis=1)
jpd2.head()

In [ ]:
# 因為屬於司法程序公告的判決無法進行分析，因此這邊刪除此類資料
indexNames = jpd2[(jpd2[4] == "司法程序公告")].index
jpd2.drop(indexNames , inplace=True)

In [ ]:
jpd2.head(10)

In [ ]:
indexNames = jpd2[(jpd2[4] == "不受理")].index
jpd2.drop(indexNames , inplace=True)
indexNames = jpd2[(jpd2[4] == "駁回")].index
jpd2.drop(indexNames , inplace=True)
indexNames = jpd2[(jpd2[4] == "撤銷")].index
jpd2.drop(indexNames , inplace=True)
indexNames = jpd2[(jpd2[4] == "無罪")].index
jpd2.drop(indexNames , inplace=True)
indexNames = jpd2[(jpd2[4] == "判決待查")].index
jpd2.drop(indexNames , inplace=True)
indexNames = jpd2[(jpd2[4] == "免訴")].index
jpd2.drop(indexNames , inplace=True)
indexNames = jpd2[(jpd2[4] == "免刑")].index
jpd2.drop(indexNames , inplace=True)

In [ ]:
jpd2[4].value_counts()

In [ ]:
# 輸出成csv檔
jpd2.to_csv('./onlyBillText2.csv', index=False, header=False, encoding='utf_8_sig')

In [ ]:
jpd2